# Report on Data Wrangling for WeRateDogs

This report shows the wrangling efforts for the data from WeRateDogs twitter account. This data was first gathered from three different sources, assessed visually and programatically, and then cleaned so that it is more useful for gaining insights and making visualizations.

## Gathering

1. We read `twitter_archive_enhanced.csv` file into a dataframe. This file contains the tweet data from WeRateDogs account, icluding the tweet text, timestamp, and expanded URL.
2. We downloaded the image predictions file provided by the course using the `get` method from the `requests` library. Then, we used the `StringIO` library to convert the contents of the response into a file like object, which we then read into a dataframe using pandas `read_csv` function.
3. We used the Twitter API to query for more data on each tweet in our dataset using its tweet id. We stored the results in a Json file whic we then read into a dataframe using pandas `read_json` function. We were only interested in the `id`, `retweet_count`, and `favorite_count` in this dataset, so we dropped the rest.

## Assessing

We found a number of issues while examining the dataset. These issues can be subdivided into quality and tidiness issues.

### Quality Issues
#### General
* Completeness: the number of entries in the three dataframes `image_predictions`, `twitter_archive`, and `twitter_stats` wasn't the same.

#### `twitter_archive` table
* The `timestamp` column had wrong dtype (`object`). It should be (`datetime`).
* Some ratings were extracted incorrectly (for example 5/10 istead of 13.5/10).
* Sometimes the wrong value was extracted as a rating (for example dates like 9/11).
* Some ratings were for groups of dogs rather than one dog. Those had a denominator that was 10x the number of dogs.
* Some rows contained retweets and replies, which we aren't interested in as they aren't dog ratings.
* Some rows contained empty values in the `expanded_urls` column.
* The `expanded_url` column sometimes contained more than one url.
* Some rows contained invalid values (links that weren't twitter photos) in the `expanded_urls` column.
* Some rows contained invalid values for the `name` column (such as 'a', 'by', and 'the').

#### `image_predictions` table
* The `p1`, `p2`, and `p3` columns had wrong dtype (`object`). They should be of dtype (`category`).

### Tidiness Issues
* Data for dogs and tweets should be divided into seperate dataframes.

#### `twitter_archive` table
* The `doggo`, `floofer`, `pupper`, and `puppo` columns were values rather than variables.

#### `image_predictions` table
* `p1`, `p2`, and `p3` were the same variable present in three different columns. The same goes for `p1_conf`, `p2_conf`, and `p3_conf` as well as `p1_dog`, `p2_dog`, and `p3_dog`.

## Cleaning

This section describes how the issues found above were addressed.

#### `image_predictions`: the `p1`, `p2`, and `p3` columns had wrong dtype (`object`). They should be of dtype (`category`).
#### `twitter_archive`: the `timestamp` column had wrong dtype (`object`). It should be (`datetime`).

##### Solution
We changed the incorrect column datatypes into the appropriate ones using pandas `astype` function.

#### `twitter_archive`: some ratings were extracted incorrectly (for example 5/10 istead of 13.5/10).
#### `twitter_archive`: sometimes the wrong value was extracted as a rating (for example dates like 9/11).

##### Solution
* To solve that ratings with fractional values were not extracted correctly, we wrote a new regex to extract whole-number as well as fractional-number ratings.
* To solve that sometimes an incorrect value was extracted as a rating, we observed that the correct rating was always at the end of the tweet. That's why we decided to use the last match. The maximum number of matches found was 3. We extracted all gruops that matched the regex, and used the last one as our rating.
* Some of the extracted values were still not dog ratings (e.g. 24/7), so we dropped the values whose denominator was indivisible by 10.

#### `twitter_archive`: some ratings were for groups of dogs rather than one dog. Those had a denominator that is 10x the number of dogs.

##### Solution
We recalculated all the rating numerators to be out of 10, then we set all the denominators to 10.

#### `twitter_archive`: some rows contained retweets and replies, which we weren't interested in as they aren't dog ratings.

##### Solution
We identified that retweets had non-empty values in `retweeted_status_id` column, so we dropped those rows. Same goes for replies and `in_reply_to_status_id` column. Afterwards we dropped all columns related to retweets and replies as they were not needed anymore.

#### `twitter_archive`: some rows contained empty values in the `expanded_urls` column.

##### Solution
Some tweets didn't contain photos, which resulted in those rows having empty values in `expanded_urls` column. Those were dropped.

#### `twitter_archive`: some rows contained invalid values for the `name` column (such as 'a', 'by', and 'the').

##### Solution
* We noticed that all valid dog names begin with capital letters, so we replaced the names that begins with small letters with `np.NaN`.
* We replaced `None` with `np.NaN`.
* We manually replaced `O` with `O'Malley`.

#### `twitter_archive`: the `doggo`, `floofer`, `pupper`, and `puppo` columns were values rather than variables.

##### Solution
To make the dataframe more tidy, we decided to replace the four value columns with one variable column called `dog_rank`. It was observed that some rows had `None` in all four columns, while others had one or more columns which weren't `None`. Using `melt()` directly to transform the four columns into rows would leave us unable to correctly identify and drop duplicates. That's why we first assigned a temporary rank of `doge` to rows who didn't have a rank, used `melt()`, and then dropped rows which had `None` in the `dog_rank` column. Finally, we replaced the temorary rank `doge` with `None`.

#### `image_predictions`: `p1`, `p2`, and `p3` were the same variable present in three different columns. 
#### The same goes for `p1_conf`, `p2_conf`, and `p3_conf` as well as `p1_dog`, `p2_dog`, and `p3_dog`.

##### Solution
We made a temporary dataframe for each prediction of the three, then we concatenated them into one big dataframe.

#### Data for dogs and tweets should be divided into seperate dataframes.

##### Solution
Since tidiness guidelines require that each unit of observation be a separate dataframe, we decided to have one dataframe for tweets and another for dogs.

* We merged `twitter_stats` dataframe onto `twitter_archive`.
* We made a new dataframe `dog_table` by merging `tweet_id`, `rating_numerator`, `rating_denominator`, `name`, and `dog_rank` columns from `twitter_archive` onto `image_predictions`.
* We dropped `rating_numerator`, `rating_denominator`, `name`, `dog_rank` columns from the tweets dataframe so that we don't have duplicate columns in multiple dataframes.